In [1]:
import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt
from scipy import sparse
import os

from setup_networks import network_from_txt, network_indices, Network, triangular_lattice_pts, get_edges, network_from_edges_and_nodes
from currents import * 
from adaptation import adaptation_ode, ss_solve
from measures import steady_state_dissipation, area_penalty, cost
from phase_diagrams import get_sinks, remove_edges, make_ellipse_netw, netw_to_nx

In [2]:
ellipse_ratio = 1.0
insertion_point_str = 'center'

N_nodes = 10000
edge_len = 0.08
N_kappas = 10;
N_rhos = 10;
num_replicates = 10;

kappas = np.logspace(-3, 0, N_kappas)
rhos = np.logspace(0, 2, N_rhos)

In [3]:
'''
Given a Network object and a ratio of the two ellipse axes (1.0 if circle), returns
the square root of the area of the network.
'''
def sqrt_area_of_network(netw, ellipse_ratio):
    inds = network_indices(netw)
    left_idx = inds['left']
    dist_fn = lambda x: np.linalg.norm(x[left_idx] - x, axis=1)
    dists = dist_fn(netw.pos)
    long_axis = np.max(dists) / 2
    short_axis = long_axis*ellipse_ratio
    return np.sqrt(np.pi * long_axis * short_axis)

'''
Arguments:
    - netw: Network object
    - K: list of conductances for each edge in the network
    - insertion_point: string ('center' or 'left') indicating the position of the source in the network
Returns: 
    - unweighted_path_length: list of lengths of shortest paths from the source to each branch point in the network, 
        where the length is measured in the number of nodes
    - path_length: list of lengths of shortest paths from the source to each branch point in the network, 
        where the length is measured in the number of nodes and paths are weighted by 1/conductance
    - path_weight: list of lengths of shortest paths from the source to each branch point in the network, 
        where the length is measured in 1/conductance and paths are weighted by 1/conductance
'''
def distance_insertion_to_branch_points(netw, K, insertion_point='center'):
    clipped_netw, clipped_K = remove_edges(netw, K)
    G = netw_to_nx(clipped_netw, clipped_K)
    
    inds = network_indices(clipped_netw)
    source = inds[insertion_point]
    
    degrees = np.array(list(G.degree))
    branch_points = np.where(degrees[:, 1] > 2)[0]
    
    unweighted_path_length = []
    path_length = []
    path_weight = []
    for b in branch_points:
        path = nx.dijkstra_path(G, source, b, weight='K')
        pathweight = nx.path_weight(G, path, weight='K')
        path_unweighted = nx.dijkstra_path(G, source, b, weight=1.)
        
        path_length += [len(path)-1] #subtract 1 to get number of edges rather than number of nodes in path
        unweighted_path_length += [len(path_unweighted) - 1]
        path_weight += [pathweight]
        
        
    return np.array(unweighted_path_length, dtype=float), np.array(path_length, dtype=float), np.array(path_weight, dtype=float)

In [28]:
# Make two arrays:
# n_kappas x n_rhos
# n_kappas x n_rhos x 

In [4]:
def rebuild_netw(N_nodes, edge_len, ellipse_ratio):
    nodes = triangular_lattice_pts(N_nodes, edge_len)
    edges = get_edges(nodes)
    netw_ = network_from_edges_and_nodes(edges, nodes)
    netw = make_ellipse_netw(netw_, 0.5, 0.5)
    netw = make_ellipse_netw(netw, 1.0, ellipse_ratio)
    return netw

In [ ]:
fig, axs = plt.subplots(N_kappas, N_rhos, figsize=(35, 25))
min_dist_to_branch_pt = np.zeros((N_kappas, N_rhos))

netw = rebuild_netw(N_nodes, edge_len, ellipse_ratio)
sqrt_area = sqrt_area_of_network(netw, ellipse_ratio)
#filepath = f'../data/old_05_23_24/expgrowth_el{ellipse_ratio}_ip{insertion_point_str}_rounded/Ks/K_'
filepath = f'../data/expgrowth_el{ellipse_ratio}_ip{insertion_point_str}/Ks/K_'

for kk in range(len(kappas)):
    k = kappas[kk]
    for pp in range(len(rhos)):
        p = rhos[pp]
        filename_ = filepath + f'kappa{np.round(k, 5)}_rho{np.round(p, 5)}'
        
        
        min_dists = 0
        path_lens = []
        for r in range(num_replicates):
            filename = filename_ + f'_replicate{r}.txt'
            if os.path.isfile(filename):
                print(f'k: {np.round(k, 5)}, p: {np.round(p, 5)}')
                K = np.loadtxt(filename)

                # compute lists of shortest path lenghts from source to branch points
                unweighted_path_lengths, path_lengths, path_weights = distance_insertion_to_branch_points(netw, K, insertion_point='center')

                 #normalize the lengths 
                path_lengths *= edge_len / sqrt_area
                unweighted_path_lengths *= edge_len / sqrt_area

                min_dists += np.sort(path_lengths)[1]
                path_lens.extend(list(path_lengths))
            
        min_dist_to_branch_pt[kk, pp] = min_dists/num_replicates
        axs[kk, pp].hist(path_lens, density=True)
            
            
            
            
            

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(35, 10))


im0 = ax.imshow(min_dist_to_branch_pt, origin='lower')
ax.set_xlabel(r'$\log_{10}(1 + \rho)$', fontsize=30);
ax.set_ylabel(r'$\log_{10}(\kappa)$', fontsize=30);
ax.set_xticks(ticks=np.arange(N_rhos), labels=np.round(np.log10(1 + rhos), 2), fontsize=20);
ax.set_yticks(ticks=np.arange(N_kappas), labels=np.round(np.log10(kappas), 2), fontsize=20);
ax.set_title('Shortest path to branch point', fontsize=30)
clb = fig.colorbar(im0, ax=ax);
clb.ax.tick_params(labelsize=20) 